In [2]:
class credit_fraud:
    def __init__(self,data):
        
        data.columns = [i.replace(' ','_').lower() for i in data.columns]
        self.raw_data = data
        
        
        
#         self.settings = settings
#         self.asset = Asset(symbol = settings['asset']['ticker'], 
#                            START = settings['START'],
#                            END = settings['END'],
#                            name = settings['asset']['name'],
#                            database = settings['asset']['database']) 
        
#         self.OM1 = OM(symbol = settings['OM1']['ticker'], 
#                       START = settings['START'],
#                       END = settings['END'],
#                       name = settings['OM1']['name'],
#                       database = settings['OM1']['database'],
#                       asset = self.asset.ticker,
#                       correlation = settings['OM1']['correlation'],
#                       beta_adjust = settings['OM1']['beta_adjust'])
        
#         self.OM2 = OM(symbol = settings['OM2']['ticker'], 
#                       START = settings['START'],
#                       END = settings['END'],
#                       name = settings['OM2']['name'],
#                       database = settings['OM2']['database'],
#                       asset = self.asset.ticker,
#                       correlation = settings['OM2']['correlation'],
#                       beta_adjust = settings['OM2']['beta_adjust'])
        
#         self.goodDays = self.goodDays()
#         self.meanCandles = None
#         self.props = None
#         self.Signals = None     
#         self.cluster_states = None
#         self.CLUSTER_LABELS = None
#         self.Neutral_Days = None
        
#         self.labels = []
#         self.longPatterns = []
#         self.shortPatterns = []
#         self.feature_set = []
        
#         # Constants #
#         self.START = self.settings['START']
#         self.END = self.settings['END']
        
#         self.IN_SAMPLE_START = self.START
#         self.IN_SAMPLE_END = "2017-01-01"

#         self.OUT_SAMPLE_START =self. IN_SAMPLE_END
#         self.OUT_SAMPLE_END = self.END
        
#         self.COLORS = {'0': 'firebrick', '1': 'green', '2': 'lightskyblue', '3': "gold"}
#         self.THRESHOLD = 0.08
#         self.NEUTRAL_COUNT = 20
#         self.NEUTRAL_THRESHOLD = 0.001
#         self.N_CLUSTERS = self.settings['n_clusters']
#         self.OPTIMIZE = self.settings['optimize']    
            

    def visualize(self):
        
        print(f"The dimensions of the raw data are {self.raw_data.shape}")
        print("The head of the raw data is as follows:")
        display(self.raw_data.head(10))
        
        print("\n")
        print("The null cases and type of each column are as follows:")
        print("\n")
        self.raw_data.info()
        
        print("\n")
        
        print("The statistics of each column are as follows:")        
        display(self.raw_data.describe().apply(lambda s: s.apply(lambda x: format(x, 'g'))))
        
        print("\n")
        print("The standard deviation and range of each feature is as follows")
        
        stds = pd.DataFrame(self.raw_data.describe().loc['std',self.raw_data.columns[1:-1]])
        data_range = pd.DataFrame(self.raw_data.describe().loc['max',self.raw_data.columns[1:-1]] - self.raw_data.describe().loc['min',self.raw_data.columns[1:-1]])
        
        fig, (ax1, ax2) = plt.subplots(2,1)
        fig.set_size_inches(20,10)

        ax1.scatter(x=np.arange(len(stds)),y=stds)
        ax1.set_ylabel('Standard Deviation')
        ax2.set_xlabel('Observations')
        ax1.set_title('Standard Deviation of the Features')
        ax1.grid()

        ax2.scatter(x=np.arange(len(data_range)),y=data_range)
        ax2.set_ylabel('Range')
        ax2.set_xlabel('Observations')
        ax2.set_title('Range of the Features')
        ax2.grid()

        plt.show()    
        
        
    def value_breakdown(self,column):
        
        column_values = self.raw_data[column]
        display(column_values.value_counts())
#         display(column_values.nunique())
        
    def plot_histograms(self,size,rows,cols,num_bins,scale):
        
        fig=plt.figure(figsize=(size,size))
        for idx, col in enumerate(self.raw_data.columns):
            ax=fig.add_subplot(rows,cols,idx+1)
            self.raw_data[col].hist(bins=num_bins,ax=ax,facecolor='navy')
            ax.set_title(col+" Distribution",color='Black')
            ax.set_yscale(scale)
        fig.tight_layout()  
        plt.show()
        
    def plot_boxplots(self,size,rows,cols):
        
        fig=plt.figure(figsize=(size,size))
        for idx, col in enumerate(self.raw_data.columns):
            ax=fig.add_subplot(rows,cols,idx+1)
            self.raw_data.boxplot(column=col,ax=ax)
            ax.set_title(col+" Box Plot",color='Black')
        fig.tight_layout()  
        plt.show()
        
    def plot_corr_matrix(self,corr_type,precision):
        
        corr = self.raw_data.corr(method=corr_type).round(precision)
        display(corr.style.background_gradient(cmap='coolwarm').set_precision(precision))
        
    def time_series_plots(self):
        fig=plt.figure(figsize=(20,20))
        for idx, col in enumerate(self.raw_data.columns[2:]):
            ax=fig.add_subplot(8,4,idx+1)
            plt.plot(self.raw_data[self.raw_data.columns[0]],self.raw_data[col])
            ax.set_title(col+" Time Series",color='Black')
            ax.set_xlabel("Time")
        fig.tight_layout()  
        plt.show()
        
    def scatter_plot(self,x,y,hx,wx):
        
        y1 = self.raw_data[x]
        x1 = self.raw_data[y]
        x_const = sm.add_constant(x1)
        model = sm.OLS(y1,x_const)
        results = model.fit()
        results.params
        
        fig, ax1 = plt.subplots()
        fig.set_size_inches(wx,hx)

        ax1.scatter(x1,y1)
        ax1.plot(x1,results.params[1] * x1 + results.params[0],color='red')
        ax1.set_ylabel('{}'.format(y))
        ax1.set_xlabel('{}'.format(x))
        ax1.set_title('{} vs {}'.format(x,y))
        ax1.grid()
        plt.show()
        
    def scatter_hist(self,x,y,xLeft,xRight,yBottom,yTop,nbins=200):
        
        left, width = 0.1, 2
        bottom, height = 0.1, 1
        spacing = 0.005


        rect_scatter = [left, bottom, width, height]
        rect_histx = [left, bottom + height + spacing, width, 0.2]
        rect_histy = [left + width + spacing, bottom, 0.2, height]

        # start with a square Figure
        fig = plt.figure(figsize=(8, 8))

        ax = fig.add_axes(rect_scatter)
        ax_histx = fig.add_axes(rect_histx, sharex=ax)
        ax_histy = fig.add_axes(rect_histy, sharey=ax)

        # no labels
        ax_histx.tick_params(axis="x", labelbottom=False)
        ax_histy.tick_params(axis="y", labelleft=False)

        y1 = self.raw_data[y]
        x1 = self.raw_data[x]

        x_const = sm.add_constant(x1)
        model = sm.OLS(y1,x_const)
        results = model.fit()
        results.params

        # the scatter plot:
        ax.scatter(x1, y1,alpha=0.1)
        ax.plot(x1,results.params[1] * x1 + results.params[0],color='red')

        # now determine nice limits by hand:
        binwidth = 0.25
        xymax = max(np.max(np.abs(x1)), np.max(np.abs(y1)))
        lim = (int(xymax/binwidth) + 1) * binwidth

        bins = np.arange(-lim, lim + binwidth, binwidth)
        ax_histx.hist(x1, bins=200)
        ax_histy.hist(y1, bins=200, orientation='horizontal')

        ax.grid()
        ax.set_xlabel(x,fontsize=15)
        ax.set_ylabel(y,fontsize=15)
        ax.set_xlim(left=xLeft,right=xRight)
        ax.set_ylim(bottom=yBottom,top=yTop)

        ax_histx.set_title("Scatter Plot of v21 vs v22 with Histograms",fontsize=18)

        plt.show()
        
    def check_duplicates(self):
        
        raw_data_copy = self.raw_data.copy()
        dups_dropped = self.raw_data.drop_duplicates(subset=None)
        print(f"The dimensions of the original data set is {self.raw_data.shape}")
        print(f"The dimensions of the data set with duplicates dropped is {dups_dropped.shape}")
        
    def remove_duplicates(self):
        
        self.raw_data.drop_duplicates(subset=None,inplace=True)
        print(f"The dimensions of the data set after the duplicates have been removed is {self.raw_data.shape}")
        
    def format_data(self,standard,robust,poly,poly_deg,test_percent):
        
#       split the data into features and target  
        self.feats = self.raw_data[self.raw_data.columns[:-1]]
        self.target = self.raw_data[self.raw_data.columns[-1]]
        
        if standard == True:
            scaler = StandardScaler()
            
            self.feats['scaled_time'] = pd.DataFrame(scaler.fit_transform(self.feats['time'].values.reshape(-1,1)))
            self.feats['scaled_amount'] = pd.DataFrame(scaler.fit_transform(self.feats['amount'].values.reshape(-1,1)))
            self.feats.drop(columns = ['time','amount'], inplace=True)
                
        elif robust == True:
            r_scaler = RobustScaler()
            
            self.feats['scaled_time'] = pd.DataFrame(r_scaler.fit_transform(self.feats['time'].values.reshape(-1,1)))
            self.feats['scaled_amount'] = pd.DataFrame(r_scaler.fit_transform(self.feats['amount'].values.reshape(-1,1)))
            self.feats.drop(columns = ['time','amount'], inplace=True)
                 
#       split into train and test data regardless of how the data has been transformated 
        self.X_test,self.y_test,self.X_trainVal,self.y_trainVal = \
            split_data(test_size=test_percent,val_size=0.2, x_features=self.feats,y_target=self.target)
            
        if poly == True:
            poly = PolynomialFeatures(degree = poly_deg, interaction_only=False, include_bias=True)
            self.X_poly_trainVal = pd.DataFrame(poly.fit_transform(self.X_trainVal))
            self.X_poly_test = pd.DataFrame(poly.fit_transform(self.X_test))
        
        
    def __repr__(self):
        return("Test")